```{toggle}
Outline

- intro
    - Overview - Why do we need a memory manager
    - overview of all the chapters

- physmem  
    - how program laid out - peter - address space and program loading
    - relocation
    - compaction
    - challenges 

- virt-seg
    - picture of MMU - sends physical address over bus
    - segmentation hardware - translation
    - what is virtual memory
    - physical mem management
    - swapping
    - compaction 
    - keeping track of free memory
    - challenges

- virt-paging
    - MMU with some mapping hardware - same picture, but now table
    - page frames and virtual pages
           - keeping track of which memory used
    - mapping & notion of faults
    - page table entry
    - page table organizations:
	   - single level.
	   - multi-level page table. 
           - inverted
    - TLB
           - global, local, tagged TLB
           - software managed TLB
    - contrast to segmentation, paging of blocks... rather than swapping entire program
    - what we use for:
           - anonymous - program, 
           - file backed - mention file caching
           - page fault
    - concluding remarks
           - once you have multi-level, massive address space. 
           - until now, described HW, and the benefits of having paged virtual memory, next chapters describe some of the key policies, mechanisms 


- reclamation
    - describe how we got here ... we run out of memory, ... using 
    - describe all the policies
    - FIFO
    - CLOCK
    - belady anomaly 
    - ... all from peter

- page-sizes
    - why big, why small, supporting multiple

- buffer-cache
    - most memory used. 

- pagefaults
    - anonymous - zero filled first access.. 
    - file backed 
    - COW

- misc
    - sharing-memory
        - sharing page frames
        - sharing page table pages
    - pinning memory

- realworld
    - Linux implementation

- concl
    - Review of MM
```

(cont:mm:intro)=
# Introduction

Intro  should talk about each of these
- [physmem](cont:mm:physmem)
- [virt-seg](cont:mm:virt-seg)
- [virt-paging](cont:mm:virt-paging)
- [reclamation](cont:mm:reclamation)
- [page-size](cont:mm:page-size)
- [buffer-cache](cont:mm:buffer-cache)
- [page-fault-handling](cont:mm:page-fault-handling)
- [pagefaults](cont:mm:pagefaults)
- [misc](cont:mm:misc)
- [realworld](cont:mm:realworld)
- [concl](cont:mm:concl)
